### Functions implemented 

A complete case k-nearest neighbour classifier, returns an estimated label for the test point:
```python
def complete_case_kNN_classifier(x_test, X_train, Y_train, O_train)
```

The HAM classifier is implemented, which is given an estimate $\hat{\Omega}$ of (or the true) $\Omega^\star$, and returns an estimated label of the test point. An estimate $\hat{\Omega}$ can be obtained from the training data using the second function:
```python
def HAM_classifier(x_test, Omega_hat, X_train, Y_train, O_train, alpha, betas, gammas)
def estimate_Omega_star(X_train, Y_train, O_train, alpha, betas, gammas)
```

The HAM classifier make use of three auxiliary functions:
```python
def transform_j_to_omega(j_omega, d)
def transform_omega_to_j(omega, d)
def all_obs_patterns_of_length(d)
```

A cv algorithm estimates values for $\alpha, \boldsymbol{\beta}, \boldsymbol{\gamma}$ using cross-validation and returns these in a list $[\hat{\alpha},\hat{\boldsymbol{\beta}}, \hat{\boldsymbol{\gamma}}]$:
```python
def estimate_params_CV(X_train, Y_train, O_train,L=5)
```

# Complete case kNN

In [ ]:
"""
complete case k-nearest neighbour classifier
"""
        
        
''' Predict value of test point using all available cases '''
def complete_case_kNN_classifier(x_test, X_train, Y_train, O_train):
    mc_distances = []
    mc_y = []
    no_matching_cases = 0
    
    for i in range(N_train):
        if np.all(np.ones(d)<=O_train[i,:]):
            no_matching_cases += 1
            dist = 0
            for j in range(d):
                dist += (x_test[j]-X_train[i,j])**2
            mc_distances.append(np.sqrt(dist))
            mc_y.append(Y_train[i])
            
    sort_index = np.argsort(mc_distances)
    k_mc = int(np.rint(1/3 * no_matching_cases**(1/(3+d))))
    
    if no_matching_cases==0:
        if np.random.uniform()<1/2:
            return 0
        else:
            return 1
    else:
        eta_sum = 0
        for i in range(k_mc):
            eta_sum += 1/k_mc * mc_y[sort_index[i]]

        if eta_sum<1/2:
            return 0
        else:
            return 1

# HAM

### Auxiliary functions

In [ ]:
""" 
NB: (0,1,0,0,1,0,1) will be the 0100101_2-th = 37_10-th entry. Can transform by taking 
omega = (omega_1,...,omega_d-1,omega_d) --> corresponding j=omega_d+2*omega_d-1+4*omega_d-2+...+2^(d-1)*omega_1
and also in the other direction
omega_d = j mod2, omega_d-1=((j-omega_d)/2) mod2, omega_d-2 = ((j-omega_d-omega_d-2)/4) mod2, et cetera.
This is implemented in the next two functions.
"""
def transform_j_to_omega(j_omega, d):
    omega=np.zeros(d,dtype="int")
    j=0
    j+=j_omega
    
    for k in range(d):
        omega[d-1-k]=j%2
        j=(j-omega[d-1-k])/2
    return omega

def transform_omega_to_j(omega, d):
    j=0
    for k in range(d):
        j+=omega[d-1-k]*(2**k)
    return j 


''' To create a list with all possible observation patterns we firstly define the following functions '''
def all_obs_patterns_of_length(d):
    pattern_list = []
    omega=np.zeros(d,dtype="int")

    for k in range(d+1):
        pattern_list.append([])
        
    for j in range(2**d):
        omega = transform_j_to_omega(j,d)
        o = int(np.sum(omega))
        pattern_list[o].append(omega)
        
    return pattern_list

    
""" Create a list with all possible observation patterns of length l with o observations (as np arrays), sorted by the number of observed values """
obs_patterns_with_o_observations_and_length_d = []
obs_patterns_with_o_observations_and_length_d.append([])
for l in range(1,d+1):
    obs_patterns_with_o_observations_and_length_d.append(all_obs_patterns_of_length(l))

### HAM
Oracle if given the true $\Omega$ instead of $\hat{\Omega}$, otherwise $\hat{\Omega}$ needs to be estimated from the data (see below)

In [ ]:
''' Predict value of fully observed test point using missing data anova decomposition '''
def HAM_classifier(x_test, Omega_hat, X_train, Y_train, O_train, alpha, betas, gammas):
    
    # f_0 is the expectation of Y-1/2
    f_hat_0 = sum(Y_train)/len(Y_train) - 1/2
    
    """ Estimate f_omega at test point for each observation pattern """   
    f_hat_omega = np.zeros(2**d)
    f_hat_omega[0] = f_hat_0 
    
    ''' Iterate through the different observation patterns '''
    for o in range(1,d+1):
        l_max = len(obs_patterns_with_o_observations_and_length_d[d][o])
        for omega_d_o in range(l_max):
            
            # get observation pattern, transform to index
            omega = obs_patterns_with_o_observations_and_length_d[d][o][omega_d_o]
            j_omega = transform_omega_to_j(omega,d)
            
            # calculate number of available cases
            N_available = 0
            for it in range(N_train):
                if (omega<=O_train[it,:]).all():
                    N_available += 1
            
            # calculate 
            if N_available>0:
                beta_omega = np.min(betas+(omega-1)*(-1000))
                gamma_omega = np.min(gammas+(omega-1)*(-1000))
                if gamma_omega==100:
                    exponent = beta_omega/(2*beta_omega+sum(omega))
                else:
                    exponent = beta_omega*gamma_omega/(gamma_omega*(2*beta_omega+sum(omega))+alpha*beta_omega)
                k_omega = 1+int(np.floor(N_available**(2*exponent)))
                        
            # Estimate f_omega at test point
            if N_available>0:

                # compute distances to test point, only considering the w-observed variables
                distances = []
                for i in range(N_train):
                    if (omega<=O_train[i,:]).all():
                        dist = 0
                        for k in range(d):
                            if omega[k]==1:
                                dist += (x_test[k]-X_train[i,k])**2
                        distances.append(np.sqrt(dist))
                    else:
                        distances.append(10000.0) # make sure that unavailable data points are not a nearest neighbour

                # sort distances
                sort_index = np.argsort(distances)

                # add Monte Carlo estimate of eta to f_omega
                for i in range(k_omega):
                    f_hat_omega[j_omega] += 1/k_omega * Y_train[sort_index[i]] 

                # substract 1/2 from the estimate
                f_hat_omega[j_omega] -= 1/2

                # substract Monte Carlo estimates of f_{omega_subset} from f_omega
                omega_subset = np.zeros(d,dtype="int")
                for j_short in range(2**o-1):
                    omega_short = transform_j_to_omega(j_short,o)
                    omega_subset = 0*omega_subset
                    omega_subset[np.nonzero(omega)] = omega_short
                    j_subset = transform_omega_to_j(omega_subset,d)

                    f_hat_omega[j_omega] -= f_hat_omega[j_subset]
  
        
    ''' Estimate eta '''
    eta_test = 1/2
    for o in range(d+1):
        l_max = len(obs_patterns_with_o_observations_and_length_d[d][o])
        for omega_d_o in range(l_max):
            
            # get observation pattern, transform to index
            omega = obs_patterns_with_o_observations_and_length_d[d][o][omega_d_o]
            j_omega = transform_omega_to_j(omega,d)
            
            # check if omega<= element for some element in Omega_hat, return 1 if this is the case
            helpy = 0 
            for elem in Omega_hat:
                if (omega<=elem).all():
                    helpy = 1
                    
            # if intersection is empty, add omega to Omega_hat
            if helpy == 1:
                eta_test += f_hat_omega[j_omega]
                            
    if eta_test<1/2:
        return 0
    else:
        return 1

### Estimate the relevant terms in the anova decomposition

In [ ]:
''' Estimate Omega_star using missing data anova decomposition '''
def estimate_Omega_star(X_train, Y_train, O_train, alpha, betas, gammas):
    
    # Quantities needed for thresholding
    tau_omega = np.ones(2**d)
    sigma_hat_omega_sq = np.zeros(2**d)
    
    # f_0 is the expectation of Y-1/2
    f_hat_0 = sum(Y_train)/len(Y_train) - 1/2
    sigma_hat_omega_sq[0] = f_hat_0**2
    
    """ Estimate f_omega at all training points for each observation pattern """   
    f_hat_omega = np.zeros((N_train,2**d))
    f_hat_omega[:,0] = f_hat_0*np.ones(N_train) 
    
    ''' Iterate through the different observation patterns '''
    for o in range(1,d+1):
        l_max = len(obs_patterns_with_o_observations_and_length_d[d][o])
        for omega_d_o in range(l_max):
            
            # get observation pattern, transform to index
            omega = obs_patterns_with_o_observations_and_length_d[d][o][omega_d_o]
            j_omega = transform_omega_to_j(omega,d)
            
            # calculate number of available cases
            N_available = 0
            for it in range(N_train):
                if (omega<=O_train[it,:]).all():
                    N_available += 1
            
            # calculate 
            if N_available>0:
                beta_omega = np.min(betas+(omega-1)*(-1000))
                gamma_omega = np.min(gammas+(omega-1)*(-1000))
                if gamma_omega==100:
                    exponent = beta_omega/(2*beta_omega+sum(omega))
                else:
                    exponent = beta_omega*gamma_omega/(gamma_omega*(2*beta_omega+sum(omega))+alpha*beta_omega)
                k_omega = 1+int(np.floor(N_available**(2*exponent)))
                tau_omega[j_omega] = 1/(N_available**(exponent/2))/16
            
            # For all training points that are available cases for the observation pattern
            for it in range(N_train):
                if (omega<=O_train[it,:]).all() and N_available>=1:
            
                    # compute distances to all other training points, only considering the w-observed variables
                    distances = []
                    for i in range(N_train):
                        if (omega<=O_train[i,:]).all():
                            dist = 0
                            for k in range(d):
                                if omega[k]==1:
                                    dist += (X_train[it,k]-X_train[i,k])**2
                            distances.append(np.sqrt(dist))
                        else:
                            distances.append(1000) # make sure that unavailable data points are not a nearest neighbour

                    # sort distances and decide on k_omega and the threshold tau_omega
                    sort_index = np.argsort(distances)

                    # add Monte Carlo estimate of eta to f_omega
                    for i in range(k_omega):
                        f_hat_omega[it,j_omega] += 1/k_omega * Y_train[sort_index[i]] 

                    # substract 1/2 from the estimate
                    f_hat_omega[it,j_omega] -= 1/2

                    # substract Monte Carlo estimates of f_{omega_subset} from f_omega
                    omega_subset = np.zeros(d,dtype="int")
                    for j_short in range(2**o-1):
                        omega_short = transform_j_to_omega(j_short,o)
                        omega_subset = 0*omega_subset
                        omega_subset[np.nonzero(omega)] = omega_short
                        j_subset = transform_omega_to_j(omega_subset,d)

                        f_hat_omega[it,j_omega] -= f_hat_omega[it,j_subset]
                    
            # Calculate Sobol index                   
            if N_available>0:
                sigma_hat_omega_sq[j_omega] = 1/N_available * np.sum(f_hat_omega[0:N_train,j_omega]**2)     

    """ Estimate Omega_hat """                        
    Omega_hat = []
    Omega_hat_cup_L_Omega_hat = []
    
    ''' Iterate through the different observation patterns, starting from the fully observed one '''
    for d_prime in range(d+1):
        
        l_max = len(obs_patterns_with_o_observations_and_length_d[d][d-d_prime])
        for omega_d_d_prime in range(l_max):
            
            # get observation pattern, transform to index
            omega = obs_patterns_with_o_observations_and_length_d[d][d-d_prime][omega_d_d_prime]
            
            # check if U({omega}) intersect hat_Omega is empty, return 0 if this is the case
            helpy = 0 
            for elem in Omega_hat_cup_L_Omega_hat:
                if (elem==omega).all():
                    helpy = 1
                    
            # if intersection is empty, add omega to Omega_hat and L(omega) to Omega_hat_cup_L_Omega_hat
            if helpy == 0:
                j_omega = transform_omega_to_j(omega,d)
                if sigma_hat_omega_sq[j_omega] >= tau_omega[j_omega]:
                    Omega_hat.append(omega)
                    omega_subset = np.zeros(d,dtype="int")
                    for j_short in range(2**(d-d_prime)):
                        omega_short = transform_j_to_omega(j_short,d-d_prime)
                        omega_subset = 0*omega_subset
                        omega_subset[np.nonzero(omega)] = omega_short
                        Omega_hat_cup_L_Omega_hat.append(omega_subset)
        
    return Omega_hat

### cv HAM

Cross-validation version of the HAM, optimal values for $\alpha$, each $\beta_j$, and each $\gamma_j$, from which a set of $k_\omega$s is calculated. These are returned and can then be used in the HAM classifier above. 

As a first step, a list with every possible combination of the parameters is initialised, where each list element contains: $[\alpha,(\beta_1,\ldots,\beta_d),(\gamma_1,\ldots,\gamma_d)]$.

In [ ]:
alpha_vals = [0,1/2,1,d]
beta_j_vals = [1/4,1/2,3/4,1]
gamma_j_vals = [1/2,1,2,100.0] # 100 will be coded as infinity later on

beta_vals = []
for j in range(len(beta_j_vals)):
    if d==2:
        beta_vals.append(np.array([beta_j_vals[j],beta_j_vals[j]]))
    elif d==3:
        beta_vals.append(np.array([beta_j_vals[j],beta_j_vals[j],beta_j_vals[j]]))
    elif d==4:
        beta_vals.append(np.array([beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j]]))
    elif d==5:
        beta_vals.append(np.array([beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j]]))
    elif d==7:
        beta_vals.append(np.array([beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j],beta_j_vals[j]]))
    else:
        print('Error in the initialisation of the CV values, d must be in {2,3,4,5,7}.') 

gamma_vals = []    
for j in range(len(gamma_j_vals)):
    if d==2:
        gamma_vals.append(np.array([gamma_j_vals[j],gamma_j_vals[j]]))
    elif d==3:
        gamma_vals.append(np.array([gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j]]))
    elif d==4:
        gamma_vals.append(np.array([gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j]]))
    elif d==5:
        gamma_vals.append(np.array([gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j]]))
    elif d==7:
        gamma_vals.append(np.array([gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j],gamma_j_vals[j]]))

CV_vals = []
for i in range(len(alpha_vals)):
    for j in range(len(beta_vals)):
        for k in range(len(gamma_vals)):                
            CV_vals.append([alpha_vals[i],beta_vals[j],gamma_vals[k]])

In [ ]:
''' Estimate Omega_star using missing data anova decomposition '''
def estimate_params_CV(X_train, Y_train, O_train,L=5):
    
    # Additional parameters are R (number of different values for k tried) and L (number of CV folds)
    
    ''' Cross validation parameters and errors are initialised and subsequently computed '''
    N_test_CV = int(N_train/L)
    if N_test_CV*L!=N_train:
        print('Error in CV folds')
    N_train_CV = N_train - N_test_CV
    
    CV_errors = np.zeros(len(CV_vals))  # will be used to store the smallest error in CV
    X_train_CV = np.zeros((N_train_CV,d))
    X_test_CV = np.zeros((N_test_CV,d))
    O_train_CV = np.zeros((N_train_CV,d))
    O_test_CV = np.zeros((N_test_CV,d))
    Y_train_CV = np.zeros(N_train_CV) 
    Y_test_CV = np.zeros(N_test_CV)
    
    Omega_hat = [np.ones(d,dtype="int")]
    
    for r in range(len(CV_vals)):
        for l in range(L):
            
            # Leave out part of the training data for cross validation
            X_train_CV[0:l*N_test_CV,:] = X_train[0:l*N_test_CV,:]
            X_test_CV[:,:] = X_train[l*N_test_CV:(l+1)*N_test_CV,:]
            X_train_CV[l*N_test_CV:N_train_CV,:] = X_train[(l+1)*N_test_CV:N_train,:]
            O_train_CV[0:l*N_test_CV,:] = O_train[0:l*N_test_CV,:]
            O_test_CV[:,:] = O_train[l*N_test_CV:(l+1)*N_test_CV,:]
            O_train_CV[l*N_test_CV:N_train_CV,:] = O_train[(l+1)*N_test_CV:N_train,:]
            Y_train_CV[0:l*N_test_CV] = Y_train[0:l*N_test_CV]
            Y_test_CV = Y_train[l*N_test_CV:(l+1)*N_test_CV]
            Y_train_CV[l*N_test_CV:N_train_CV] = Y_train[(l+1)*N_test_CV:N_train]
                   
            Omega_hat = [np.ones(d,dtype="int")]
        
            """ 
            Calculate test error on the left out set
            """                    
            for it in range(N_test_CV):
                x_test = X_test_CV[it,:]
                
                # f_0 is the expectation of Y-1/2
                f_hat_0 = sum(Y_train_CV)/len(Y_train_CV) - 1/2

                """ Estimate f_omega at test point for each observation pattern """   
                f_hat_omega = np.zeros(2**d)
                f_hat_omega[0] = f_hat_0 

                ''' Iterate through the different observation patterns '''
                for o in range(1,d+1):
                    l_max = len(obs_patterns_with_o_observations_and_length_d[d][o])
                    for omega_d_o in range(l_max):

                        # get observation pattern, transform to index
                        omega = obs_patterns_with_o_observations_and_length_d[d][o][omega_d_o]
                        j_omega = transform_omega_to_j(omega,d)

                        if (omega<=O_test_CV[it,:]).all():

                            # calculate number of available cases
                            N_available = 0
                            for i in range(N_train_CV):
                                if (omega<=O_train_CV[i,:]).all():
                                    N_available += 1

                            # calculate 
                            if N_available>0:
                                alpha = CV_vals[r][0]
                                beta_omega = np.min(CV_vals[r][1]+(omega-1)*(-1000))
                                gamma_omega = np.min(CV_vals[r][2]+(omega-1)*(-1000))
                                if gamma_omega!=100:
                                    exponent = beta_omega*gamma_omega/(gamma_omega*(2*beta_omega+sum(omega))+alpha*beta_omega)
                                else:
                                    exponent = beta_omega/(2*beta_omega+sum(omega))
                                k_omega = 1+int(np.floor(N_available**(2*exponent)))

                            # Estimate f_omega at test point
                            if N_available>0:

                                # compute distances to test point, only considering the w-observed variables
                                distances = []
                                for i in range(N_train_CV):
                                    if (omega<=O_train_CV[i,:]).all():
                                        dist = 0
                                        for k in range(d):
                                            if omega[k]==1:
                                                dist += (x_test[k]-X_train_CV[i,k])**2
                                        distances.append(np.sqrt(dist))
                                    else:
                                        distances.append(10000.0) # make sure that unavailable data points are not a nearest neighbour

                                # sort distances
                                sort_index = np.argsort(distances)

                                # add Monte Carlo estimate of eta to f_omega
                                for i in range(k_omega):
                                    f_hat_omega[j_omega] += 1/k_omega * Y_train_CV[sort_index[i]] 

                                # substract 1/2 from the estimate
                                f_hat_omega[j_omega] -= 1/2

                                # substract Monte Carlo estimates of f_{omega_subset} from f_omega
                                omega_subset = np.zeros(d,dtype="int")
                                for j_short in range(2**o-1):
                                    omega_short = transform_j_to_omega(j_short,o)
                                    omega_subset = 0*omega_subset
                                    omega_subset[np.nonzero(omega)] = omega_short
                                    j_subset = transform_omega_to_j(omega_subset,d)

                                    f_hat_omega[j_omega] -= f_hat_omega[j_subset]


                ''' Estimate eta '''
                eta_test = 1/2
                for o in range(d+1):
                    l_max = len(obs_patterns_with_o_observations_and_length_d[d][o])
                    for omega_d_o in range(l_max):

                        # get observation pattern, transform to index
                        omega = obs_patterns_with_o_observations_and_length_d[d][o][omega_d_o]
                        j_omega = transform_omega_to_j(omega,d)

                        # check if omega<= element for some element in Omega_hat, return 1 if this is the case
                        helpy = 0 
                        for elem in Omega_hat:
                            if (omega<=elem).all():
                                helpy = 1

                        # if intersection is empty, add omega to Omega_hat
                        if helpy == 1:
                            eta_test += f_hat_omega[j_omega]

                if eta_test<1/2 and Y_test_CV[it]==1:
                    CV_errors[r] += 1
                elif eta_test>=1/2 and Y_test_CV[it]==0:
                    CV_errors[r] += 1
                
    r_optimal = np.argmin(CV_errors)
        
    return CV_vals[r_optimal] 